In [14]:
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
import pandas as pd
import numpy as np
import json
from sklearn import preprocessing, metrics
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest, chi2

In [2]:
# file = open('Antidepressants.txt', encoding='utf-8') 
with open('all_reviews_texts.txt', encoding='UTF8') as file:
    reviews = pd.DataFrame()
    for line in file:
        pik = json.loads(line)
        reviews = reviews.append(pd.DataFrame.from_dict(pik))  

In [3]:
reviews.head(5)

,url,cat1,cat2,cat3,cat4,rating_1,rating_2,rating_3,rating_4,rating_5,...,title,author,author_location,authorid,recom_author_mark,overall,description,product_name_url,product_name_title,product-rating
0,https://otzovik.com/review_1650685.html,Красота и здоровье,Лекарственные средства,Разное,Solvay Pharma,width: 100%;,width: 100%;,width: 100%;,width: 100%;,width: 100%;,...,Отзыв: Таблетки от головокружения Бетасерк - П...,Любовная любовь,"Россия, Рязань",/profile/%D0%9B%D1%8E%D0%B1%D0%BE%D0%B2%D0%BD%...,ДА,"Помогает при головокружении, только нужно прин...",У меня последний год стала кружиться голова. Я...,https://otzovik.com/reviews/preparat_solvay_ph...,Все отзывы о Таблетки от головокружения Бетасерк,Общий рейтинг: 5
0,https://otzovik.com/review_5260629.html,В городе,Больницы и клиники,Детские больницы,,width: 100%;,width: 100%;,width: 100%;,width: 100%;,width: 100%;,...,Отзыв: Луганская областная детская клиническая...,Аноним1383147,"Украина, Луганск",/profile/%D0%90%D0%BD%D0%BE%D0%BD%D0%B8%D0%BC1...,ДА,замечательная больница.отдельное спасибо онког...,Я мать ребёнка который попал в эту больницу. и...,https://otzovik.com/reviews/luganskaya_oblastn...,Все отзывы о Луганская областная детская клини...,Общий рейтинг: 5
0,https://otzovik.com/review_6665469.html,Красота и здоровье,Лекарственные средства,Ноотропные препараты,EGIS,width: 100%;,width: 100%;,width: 100%;,width: 100%;,width: 100%;,...,"Отзыв: Ноотропное средство EGIS ""Луцетам"" - Эф...",maxzykov90,"Россия, Барнаул",/profile/maxzykov90,ДА,"Эффективно, однако нужны высокие дозы",Обычный пирацетам выпускается в максимальной д...,https://otzovik.com/reviews/nootropnoe_sredstv...,"Все отзывы о Ноотропное средство EGIS ""Луцетам""",Общий рейтинг: 5
0,https://otzovik.com/review_544236.html,Красота и здоровье,Альтернативная медицина,Гомеопатические препараты,Bionorica,width: 100%;,width: 100%;,width: 100%;,width: 100%;,,...,Отзыв: Гомеопатический препарат Циклодинон - т...,natalipon,"Россия, Нижний Новгород",/profile/natalipon,ДА,теперь мой организм работает регулярно,Циклодинон мне прописал врач для нормализации ...,https://otzovik.com/reviews/gomeopaticheskiy_p...,Все отзывы о Гомеопатический препарат Циклодинон,Общий рейтинг: 5
0,https://otzovik.com/review_3322356.html,Красота и здоровье,Альтернативная медицина,БАДы,Фелицата Холдинг,width: 20%;,width: 60%;,width: 60%;,width: 80%;,,...,Отзыв: БАД Фелицата Холдинг Прелакс - Совершен...,karines,"Украина, Ивано-Франковск",/profile/karines,,Совершенно без полезное,Всем доброго времени суток. Сегодня хочу с ва...,https://otzovik.com/reviews/bad_felicata_holdi...,Все отзывы о БАД Фелицата Холдинг Прелакс,Общий рейтинг: 2


In [3]:
reviews = reviews.replace(['width: 20%;', 'width: 40%;', 'width: 60%;', 'width: 80%;', 'width: 100%;'], [1, 2, 3, 4, 5])
reviews = reviews.replace(['Общий рейтинг: 1', 'Общий рейтинг: 2', 'Общий рейтинг: 3', 'Общий рейтинг: 4', 'Общий рейтинг: 5'], [1, 2, 3, 4, 5])

In [4]:
reviews_part = pd.DataFrame()
for i in range(1,6):
    reviews_part = reviews_part.append(reviews[reviews['product-rating']==i].head(500))
reviews_part['review'] = reviews_part['overall'].str.cat(reviews_part['description'], sep='. ') 
reviews_part.head()

,url,cat1,cat2,cat3,cat4,rating_1,rating_2,rating_3,rating_4,rating_5,...,author,author_location,authorid,recom_author_mark,overall,description,product_name_url,product_name_title,product-rating,review
0,https://otzovik.com/review_3514847.html,В городе,Больницы и клиники,Акушерство и гинекология,"Россия, Московская область",1,1,1,1,1,...,Аноним853156,"Россия, Москва",/profile/%D0%90%D0%BD%D0%BE%D0%BD%D0%B8%D0%BC8...,,Ужас! Не рожайте там,Хочу оставить свой отзыв дабы огородить от это...,https://otzovik.com/reviews/roddom_g_lyuberci_...,"Все отзывы о Роддом г. Люберцы (Россия, Люберцы)",1,Ужас! Не рожайте там. Хочу оставить свой отзыв...
0,https://otzovik.com/review_5640852.html,В городе,Больницы и клиники,Диагностические центры и лаборатории,,1,1,1,1,1,...,Кристина Вайт,"Россия, Москва",/profile/%D0%9A%D1%80%D0%B8%D1%81%D1%82%D0%B8%...,,ужасно и неккоректно,На протяжении всей беременности сдавала анализ...,https://otzovik.com/reviews/laboratoriya_gemot...,"Все отзывы о Лаборатория ""Гемотест""",1,ужасно и неккоректно. На протяжении всей берем...
0,https://otzovik.com/review_3860601.html,В городе,Больницы и клиники,Поликлиники,"Россия, Москва",1,1,1,1,1,...,Avtostop,"Россия, Москва",/profile/Avtostop,,Приёмный покой - нет слов! Одни эмоции!,Безалаберность в приёмном покое! 4.10.16 около...,https://otzovik.com/reviews/gorodskaya_klinich...,Все отзывы о Городская клиническая больница им...,1,Приёмный покой - нет слов! Одни эмоции!. Безал...
0,https://otzovik.com/review_3215475.html,Красота и здоровье,Альтернативная медицина,Гомеопатические препараты,Сашера-Мед,1,3,1,1,,...,Добрая весна,Россия,/profile/%D0%94%D0%BE%D0%B1%D1%80%D0%B0%D1%8F+...,,Всего понемножку. И с чего глистам пугаться?) ...,Капли Интоксик - новомодное отечественное сред...,https://otzovik.com/reviews/kapli_rastitelnie_...,Все отзывы о Капли растительные Сашера-Мед Int...,1,Всего понемножку. И с чего глистам пугаться?) ...
0,https://otzovik.com/review_4210625.html,В городе,Больницы и клиники,Диагностические центры и лаборатории,,3,5,4,1,1,...,кукузя1000,"Россия, Санкт-Петербург",/profile/%D0%BA%D1%83%D0%BA%D1%83%D0%B7%D1%8F1000,,"Разводят на ненужные услуги, дорого","Очень разочарована, как и почему медицинская к...",https://otzovik.com/reviews/medicinskiy_diagno...,Все отзывы о Медицинский диагностический центр...,1,"Разводят на ненужные услуги, дорого. Очень раз..."


In [5]:
# X_train, X_test, y_train, y_test = train_test_split(antigroup.drop(['url', 'cat1', 'cat2', 'cat3', 'cat4', 'rating_1', 'rating_2', 'rating_3', 'rating_4', 'rating_5','rating_1_title', 'rating_2_title', 'rating_3_title', 'rating_4_title', 'rating_5_title', 'review_date', 'recom_number', 'author', 'author_location', 'authorid', 'recom_author_mark', 'product_name_url', 'product_name_title', 'product-rating', 'Positively_Rated', 'title', 'plus', 'minus'], axis=1), antigroup['Positively_Rated'], test_size=0.2, random_state=0)
# X_train, X_test, y_train, y_test = train_test_split(reviews_part.drop(['product-rating'], axis = 1), reviews_part['product-rating'], test_size=0.2, random_state=0)
X_train, X_test, y_train, y_test = train_test_split(reviews_part['review'], reviews_part['product-rating'], test_size=0.2, random_state=0)

In [6]:
vectorizer = TfidfVectorizer(min_df = 5, max_features = 50000)
tf_transformer = TfidfTransformer(use_idf=True)
X_counts = vectorizer.fit_transform(X_train)
X_tfidf = tf_transformer.fit_transform(X_counts)

In [7]:
cla = LogisticRegression(multi_class = 'multinomial', solver='lbfgs')
clf = cla.fit(X_tfidf, y_train)
predicted = clf.predict(X_tfidf)

In [8]:
print(metrics.classification_report(y_train, predicted))
print('MSE: ', metrics.mean_squared_error(y_train, predicted))

              precision    recall  f1-score   support

           1       0.95      0.99      0.97       405
           2       0.98      0.98      0.98       401
           3       0.99      0.97      0.98       406
           4       0.98      0.98      0.98       397
           5       0.99      0.98      0.99       391

    accuracy                           0.98      2000
   macro avg       0.98      0.98      0.98      2000
weighted avg       0.98      0.98      0.98      2000

MSE:  0.0885


In [9]:
X_new_counts = vectorizer.transform(X_test)
X_new_tfidf = tf_transformer.transform(X_new_counts)
predicted = clf.predict(X_new_tfidf)

In [10]:
print(metrics.classification_report(y_test, predicted))
print('MSE: ', metrics.mean_squared_error(y_test, predicted))

              precision    recall  f1-score   support

           1       0.45      0.59      0.51        95
           2       0.21      0.14      0.17        99
           3       0.30      0.31      0.31        94
           4       0.33      0.38      0.35       103
           5       0.53      0.46      0.49       109

    accuracy                           0.38       500
   macro avg       0.36      0.38      0.37       500
weighted avg       0.37      0.38      0.37       500

MSE:  1.846


In [11]:
from sklearn.pipeline import Pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(min_df = 5, max_features = 50000)),
    ('clf', LogisticRegression(multi_class = 'multinomial', solver='lbfgs', max_iter = 1000)),
])
parameters = {
    'tfidf__use_idf': (True, False),
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'clf__C': [1e-5, 1e-3, 1e-1, 1e0, 1e1, 1e2]
}
grid_search_tune = GridSearchCV(pipeline, parameters, iid = False, cv=5)
grid_search_tune.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('tfidf',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=50000,
                                                        min_df=5,
                                                        ngram_range=(1, 1),
                                        

In [12]:
print("Best parameters set:")
print(grid_search_tune.best_params_)
print('\n', grid_search_tune.best_estimator_.steps)

Best parameters set:
{'clf__C': 1.0, 'tfidf__ngram_range': (1, 2), 'tfidf__use_idf': True}

 [('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=50000,
                min_df=5, ngram_range=(1, 2), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)), ('clf', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False))]


In [13]:
best_clf = grid_search_tune.best_estimator_.fit(X_train, y_train)
predicted = best_clf.predict(X_test)
print(metrics.classification_report(y_test, predicted))
print('MSE: ', metrics.mean_squared_error(y_test, predicted))

              precision    recall  f1-score   support

           1       0.51      0.64      0.57        95
           2       0.26      0.18      0.21        99
           3       0.30      0.37      0.33        94
           4       0.38      0.36      0.37       103
           5       0.56      0.50      0.53       109

    accuracy                           0.41       500
   macro avg       0.40      0.41      0.40       500
weighted avg       0.40      0.41      0.40       500

MSE:  1.538


# Наиболее значимые признаки

In [42]:
X_counts2 = grid_search_tune.best_estimator_.named_steps['tfidf'].fit_transform(X_train)

select_best_features = SelectKBest(chi2)
select_best_features.fit(X_counts2, y_train)

best_features = pd.DataFrame(list(zip(grid_search_tune.best_estimator_.named_steps['tfidf'].get_feature_names(), select_best_features.scores_)), 
                                       columns = ['feature', 'score'])

best_features.sort_values('score', ascending=False)

,feature,score
2340,деньги,12.094731
12596,ужас,10.796463
10322,развод,8.639598
12602,ужасное,7.904217
2298,денег,7.280068
4211,клинике,7.206685
5510,мрт,6.394038
6526,не советую,6.277149
12962,хорошее,6.172341
6448,не помог,6.113550
